In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

from sklearn.metrics import precision_score, recall_score

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, make_scorer

from sklearn.metrics import precision_recall_curve

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
source_folder='/content/drive/My Drive/B2C_df.csv'


In [ ]:
import pandas as pd
df = pd.DataFrame(pd.read_csv(source_folder, index_col = None))


In [ ]:
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
source_folder2='/content/drive/My Drive/B2B_df.csv'
import pandas as pd
df2 = pd.DataFrame(pd.read_csv(source_folder2, index_col = None))


In [ ]:
# Concatenate them one after the other
combined_df = pd.concat([df, df2], axis=0)

# Reset the index of the combined DataFrame, if necessary
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
df=combined_df.copy()
df

In [ ]:
df.info()

In [ ]:

df[['Frequency', 'Amount']] = df[['Frequency', 'Amount']].astype('float64')


In [ ]:

numerical_features = df.select_dtypes(include=['float64'])


numerical_features.boxplot(figsize=(10, 6))
plt.title('Boxplot of Numerical Features')
plt.ylabel('Value')
plt.xticks(rotation=90)
plt.show()


In [ ]:
numeric_df = df.select_dtypes(include=['float64', 'int64'])

numeric_df = pd.DataFrame(numeric_df, columns=numeric_df.columns)


plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
df_reduced=df.copy()
df_reduced['Average_Amount_per_Frequency'] = df_reduced['Amount'] / df_reduced['Frequency']

df_reduced = df_reduced.drop(['Amount', "Frequency"], axis=1)

df_reduced = df_reduced.drop(["order_channel_by phone"], axis=1)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

encoded_data = encoder.fit_transform(df_reduced[['segment']])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['segment']))

df_reduced = pd.concat([df_reduced.drop('segment', axis=1), encoded_df], axis=1)


In [ ]:
plt.figure(figsize=(16, 12))
sns.heatmap(df_reduced.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df_reduced = df_reduced.drop(["segment_hibernating"], axis=1)

In [ ]:
X = df_reduced.drop(columns=['client_id', 'Churn'])  # Drop non-feature columns
y = df_reduced['Churn']  # Target variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

# Models

## Simple Logistic Regresion

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = LogisticRegression(random_state=42)


model.fit(X_train_scaled, y_train)


y_pred = model.predict(X_test_scaled)

# Model evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(3,3))
sns.heatmap(cm, annot=True, fmt="d", ax=ax, cmap="Blues", annot_kws={"size": 20})
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Not Churn', 'Churn'])
ax.yaxis.set_ticklabels(['Not Churn', 'Churn'])

plt.show()

print("Classification Report:\n", classification_report(y_test, y_pred))

## Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

sel_casual = SelectKBest(score_func=f_regression)
sel_casual.fit(X_train, y_train)


In [ ]:
import plotly.express as px

scores = pd.Series(sel_casual.scores_, index=X_train.columns)
scores = scores.sort_values(ascending=False)
px.bar(scores, template="none", title="F-Score of features with Churn as dependent variable")

## Random Forest Model

In [ ]:
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

smote = SMOTE(random_state=42)

# Create a pipeline that first oversamples the minority class with SMOTE, then fits the RandomForest model
pipeline = Pipeline([
    ('smote', smote),
    ('classifier', rf_model)
])


param_grid = {
    'classifier__n_estimators': [20, 50],
    'classifier__max_depth': [3, 5, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2, 4]
}


f1_scorer = make_scorer(f1_score)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Setup GridSearchCV with the F1 scorer
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=cv, scoring=f1_scorer, verbose=1)

grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)

### Model Evaluation

In [ ]:

rf_f1_score = f1_score(y_test, y_pred)
rf_confusion_matrix = confusion_matrix(y_test, y_pred)
rf_classification_report = classification_report(y_test, y_pred)

print(f'F1 Score: {rf_f1_score}')
fig, ax = plt.subplots(figsize=(3,3))
sns.heatmap(rf_confusion_matrix, annot=True, fmt="d", ax=ax, cmap="Blues", annot_kws={"size": 20})
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix Random Forest')
ax.xaxis.set_ticklabels(['Not Churn', 'Churn'])
ax.yaxis.set_ticklabels(['Not Churn', 'Churn'])

plt.show()
print(f'Classification Report:\n {rf_classification_report}')

In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

#### Predict probabilities

In [ ]:

y_pred_proba = best_rf_model.predict_proba(X_test)[:, 1]

#### ROC CURVE

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)


roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('RF ROC CURVE')
plt.legend(loc="lower right")
plt.show()


#### Feature importance ranking Random Forest

In [ ]:
rf = best_rf_model.named_steps['classifier']

# Get feature importances from the random forest
feature_importances = rf.feature_importances_

feature_names = X_train.columns

sorted_indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Feature Importance Ranking Random Forest")
plt.bar(range(X_train.shape[1]), feature_importances[sorted_indices], align='center')
plt.xticks(range(X_train.shape[1]), feature_names[sorted_indices], rotation=90)
plt.tight_layout()
plt.show()

## XGBoost Model

In [ ]:
smote = SMOTE(random_state=42)

xgb_model = XGBClassifier(random_state=42, scale_pos_weight=1, use_label_encoder=False, eval_metric='logloss')

pipeline = ImbPipeline([
    ('smote', smote),
    ('xgb', xgb_model)
])


param_grid = {
    'xgb__n_estimators': [20, 50, 80],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__max_depth': [3, 5, 7],
}

# Define custom scorers for evaluation
scorers = {
    'f1_score': make_scorer(f1_score),
    'roc_auc': 'roc_auc',
    'pr_auc': make_scorer(average_precision_score, needs_proba=True)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=cv, scoring=scorers, refit='f1_score', verbose=1)

grid_search.fit(X_train, y_train)

#best estimator
best_xgb_model = grid_search.best_estimator_


y_pred = best_xgb_model.predict(X_test)
y_pred_proba = best_xgb_model.predict_proba(X_test)[:, 1]

### Model Evaluation

In [ ]:
# Calculate different metrics
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

# Generate the classification report and confusion matrix
classification_report_output = classification_report(y_test, y_pred)
confusion_matrix_output = confusion_matrix(y_test, y_pred)

print(f'Best XGB Model Parameters: {grid_search.best_params_}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')
print(f'Precision-Recall AUC Score: {pr_auc}')
print('Classification Report XGBoost:\n', classification_report_output)
fig, ax = plt.subplots(figsize=(3,3))
sns.heatmap(confusion_matrix_output, annot=True, fmt="d", ax=ax, cmap="Blues", annot_kws={"size": 20})
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix XGBoost')
ax.xaxis.set_ticklabels(['Not Churn', 'Churn'])
ax.yaxis.set_ticklabels(['Not Churn', 'Churn'])

plt.show()


In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

#### ROC CURVE

In [ ]:
y_pred_proba = best_xgb_model.predict_proba(X_test)[:, 1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calculate the Area Under the Curve (AUC)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost ROC CURVE')
plt.legend(loc="lower right")
plt.show()


### Precision and Recall Balance

In [ ]:
y_scores = best_xgb_model.predict_proba(X_test)[:, 1]

# Precision-recall curve
precision_recall_curve(y_test, y_scores)


In [ ]:
thresholds = np.linspace(0,1,1000)

In [ ]:
precs = []
recs = []
for th in thresholds:
    precs.append(precision_score(y_test.astype(float), (y_scores >= th).astype(float)))
    recs.append(recall_score(y_test.astype(float), (y_scores >= th).astype(float)))

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=thresholds,
    y=precs,
    name="Precision",
    line_width=3
))
fig.add_trace(go.Scatter(
    x=thresholds,
    y=recs,
    name="Recall",
    line_width=3
))

fig.update_layout(title="Precision-Recall-Threshold", xaxis_title="Threshold", yaxis_title="Precision / Recall", hovermode="x", template="none")